In [1]:
import sys,os,imageio
root = '/mnt/new_disk2/anpei/code/MVS-NeRF'
os.chdir(root)
sys.path.append(root)

from opt import config_parser
from data import dataset_dict
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


# models
from models import *
from renderer import *
from data.ray_utils import get_rays

from tqdm import tqdm


from skimage.metrics import structural_similarity

# pytorch-lightning
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import LightningModule, Trainer, loggers


from data.ray_utils import ray_marcher

%load_ext autoreload
%autoreload 2

torch.cuda.set_device(3)
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

# Rendering video from finetuned ckpts

In [2]:
def decode_batch(batch):
    rays = batch['rays']  # (B, 8)
    rgbs = batch['rgbs']  # (B, 3)
    return rays, rgbs

def unpreprocess(data, shape=(1,1,3,1,1)):
    # to unnormalize image for visualization
    # data N V C H W
    device = data.device
    mean = torch.tensor([-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225]).view(*shape).to(device)
    std = torch.tensor([1 / 0.229, 1 / 0.224, 1 / 0.225]).view(*shape).to(device)

    return (data - mean) / std


def normalize(x):
    return x / np.linalg.norm(x)

def viewmatrix(z, up, pos):
    vec2 = normalize(z)
    vec1_avg = up
    vec0 = normalize(np.cross(vec1_avg, vec2))
    vec1 = normalize(np.cross(vec2, vec0))
    m = np.eye(4)
    m[:3] = np.stack([vec0, vec1, vec2, pos], 1)
    return m

def ptstocam(pts, c2w):
    tt = np.matmul(c2w[:3,:3].T, (pts-c2w[:3,3])[...,np.newaxis])[...,0]
    return tt

def poses_avg(poses):

    center = poses[:, :3, 3].mean(0)
    vec2 = normalize(poses[:, :3, 2].sum(0))
    up = poses[:, :3, 1].sum(0)
    c2w = viewmatrix(vec2, up, center)
    
    return c2w


def render_path_spiral(c2w, up, rads, focal, zdelta, zrate, N_rots=2, N=120):
    render_poses = []
    rads = np.array(list(rads) + [1.])
    
    for theta in np.linspace(0., 2. * np.pi * N_rots, N+1)[:-1]:
        c = np.dot(c2w[:3,:4], np.array([np.cos(theta), -np.sin(theta), -np.sin(theta*zrate), 1.]) * rads) 
        z = normalize(c - np.dot(c2w[:3,:4], np.array([0,0,-focal, 1.])))
        render_poses.append(viewmatrix(z, up, c))
    return render_poses

def position2angle(position, N_views=16, N_rots = 2):
    ''' nx3 '''
    position = normalize(position)
    theta = np.arccos(position[:,1])/np.pi*180
    phi = np.arctan2(position[:,2],position[:,0])/np.pi*180
    return [theta,phi]

def get_spiral(c2ws_all, near_far, N_views=120):

    # center pose
    c2w = poses_avg(c2ws_all)
    
    # Get average pose
    up = normalize(c2ws_all[:, :3, 1].sum(0))

    # Find a reasonable "focus depth" for this dataset
    close_depth, inf_depth = near_far
    dt = .75
    mean_dz = 1./(((1.-dt)/close_depth + dt/inf_depth))
    focal = mean_dz

    # Get radii for spiral path
    shrink_factor = .8
    zdelta = close_depth * .2
    tt = c2ws_all[:,:3,3] - c2w[:3,3][None]
    rads = np.percentile(np.abs(tt), 70, 0)*0.5
    render_poses = render_path_spiral(c2w, up, rads, focal, zdelta, zrate=.5, N=N_views)
    return np.stack(render_poses)


def nerf_video_path(c2ws, theta_range=20,phi_range=10,N_views=120):
    mean_position = torch.mean(c2ws[:,:3, 3],dim=0).reshape(1,3).cpu().numpy()
    theta,phi = position2angle(mean_position)
    render_poses = [pose_spherical_nerf(angle+theta[0], -phi[0]-phi_range, 4.0) for angle in np.linspace(-theta_range,theta_range,N_views//4, endpoint=False)]
    render_poses += [pose_spherical_nerf(theta_range+theta[0], -phi[0]+angle, 4.0) for angle in np.linspace(-phi_range,phi_range,N_views//4, endpoint=False)]
    render_poses += [pose_spherical_nerf(angle+theta[0], -phi[0]+phi_range, 4.0) for angle in np.linspace(theta_range,-theta_range,N_views//4, endpoint=False)]
    render_poses += [pose_spherical_nerf(-theta_range+theta[0], -phi[0]+angle, 4.0) for angle in np.linspace(phi_range,-phi_range,N_views//4, endpoint=False)]
    render_poses = torch.from_numpy(np.stack(render_poses)).float().to(device)
    return render_poses

In [18]:

for i_scene, scene in enumerate(['horns']):#'flower','orchids', 'room','leaves','fern','horns','trex','fortress'
    # add --use_color_volume if the ckpts are fintuned with this flag
    cmd = f'--datadir /mnt/new_disk_2/anpei/Dataset/MVSNeRF/nerf_llff_data/{scene}  \
     --dataset_name llff --imgScale_test {1.0}  --netwidth 128 --net_type v0 '

    is_finetued = False # set False if rendering without finetuning
    if is_finetued:
        cmd += f'--ckpt ./runs_fine_tuning/{scene}/ckpts//latest.tar'
    else:
        cmd += '--ckpt ./ckpts//mvsnerf-v0.tar'
        
    args = config_parser(cmd.split())
    args.use_viewdirs = True

    args.N_samples = 128
    args.feat_dim = 8+3*4 if args.net_type!='v2' else 8+3*3
    args.use_color_volume = False if not is_finetued else args.use_color_volume

    # create models
    if 0==i_scene:
        render_kwargs_train, render_kwargs_test, start, grad_vars = create_nerf_mvs(args, use_mvs=True, dir_embedder=args.net_type=='v2', pts_embedder=True)
        filter_keys(render_kwargs_train)

        MVSNet = render_kwargs_train['network_mvs']
        render_kwargs_train.pop('network_mvs')


    datadir = args.datadir
    datatype = 'val'
    pad = 24
    args.chunk = 5120


    dataset = dataset_dict[args.dataset_name](args, split=datatype)
    val_idx = dataset.img_idx
    

    save_dir = f'results/test3'
    os.makedirs(save_dir, exist_ok=True)
    MVSNet.train()
    MVSNet = MVSNet.cuda()
    
    with torch.no_grad():

        c2ws_all = dataset.poses

        if is_finetued:   
            # large baselien
            imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(device=device)
            volume_feature = torch.load(args.ckpt)['volume']['feat_volume']
            volume_feature = RefVolume(volume_feature.detach()).cuda()
            
            pad *= args.imgScale_test
            w2cs, c2ws = pose_source['w2cs'], pose_source['c2ws']
            pair_idx = torch.load('configs/pairs.th')[f'{scene}_train']
#             c2ws_render = c2ws_all[pair_idx]
            c2ws_render = get_spiral(c2ws_all[pair_idx], near_far_source, N_views=40)
        else:            
            # neighboring views with position distance
            random_selete = torch.randint(0,len(c2ws_all),(1,))    #!!!!!!!!!! you may change this line if rendering a specify view 
            dis = np.sum(c2ws_all[:,:3,3] * c2ws_all[[random_selete],:3,3], axis=-1)
            pair_idx = np.argsort(dis)[::-1][torch.randperm(5)[:3]]
            imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(pair_idx=pair_idx, device=device)
            volume_feature, _, _ = MVSNet(imgs_source, proj_mats, near_far_source, pad=pad)
            

            c2ws_render = gen_render_path(c2ws_all[pair_idx], N_views=10)
        c2ws_render = torch.from_numpy(np.stack(c2ws_render)).float().to(device)
            #####
            # still a bug in the gen_render_path
            # you may re-run it if the video looks abnormal ...
            
        imgs_source = unpreprocess(imgs_source)

        try:
            tqdm._instances.clear() 
        except Exception:     
            pass
        
        frames = []
        img_directions = dataset.directions.to(device)
        for i, c2w in enumerate(tqdm(c2ws_render)):
            torch.cuda.empty_cache()
            
            rays_o, rays_d = get_rays(img_directions, c2w)  # both (h*w, 3)
            rays = torch.cat([rays_o, rays_d,
                     near_far_source[0] * torch.ones_like(rays_o[:, :1]),
                     near_far_source[1] * torch.ones_like(rays_o[:, :1])],
                    1).to(device)  # (H*W, 3)
            
            
            N_rays_all = rays.shape[0]
            rgb_rays, depth_rays_preds = [],[]
            for chunk_idx in range(N_rays_all//args.chunk + int(N_rays_all%args.chunk>0)):

                xyz_coarse_sampled, rays_o, rays_d, z_vals = ray_marcher(rays[chunk_idx*args.chunk:(chunk_idx+1)*args.chunk],
                                                    N_samples=args.N_samples)

                # Converting world coordinate to ndc coordinate
                H, W = imgs_source.shape[-2:]
                inv_scale = torch.tensor([W - 1, H - 1]).to(device)
                w2c_ref, intrinsic_ref = pose_source['w2cs'][0], pose_source['intrinsics'][0].clone()
                xyz_NDC = get_ndc_coordinate(w2c_ref, intrinsic_ref, xyz_coarse_sampled, inv_scale,
                                             near=near_far_source[0], far=near_far_source[1], pad=pad)


                # rendering
                rgb, disp, acc, depth_pred, alpha, extras = rendering(args, pose_source, xyz_coarse_sampled,
                                                                       xyz_NDC, z_vals, rays_o, rays_d,
                                                                       volume_feature,imgs_source, **render_kwargs_train)
    
                
                rgb, depth_pred = torch.clamp(rgb.cpu(),0,1.0).numpy(), depth_pred.cpu().numpy()
                rgb_rays.append(rgb)
                depth_rays_preds.append(depth_pred)

            
            depth_rays_preds = np.concatenate(depth_rays_preds).reshape(H, W)
            depth_rays_preds, _ = visualize_depth_numpy(depth_rays_preds, near_far_source)
            
            rgb_rays = np.concatenate(rgb_rays).reshape(H, W, 3)
            H_crop, W_crop = np.array(rgb_rays.shape[:2])//20
            rgb_rays = rgb_rays[H_crop:-H_crop,W_crop:-W_crop]
            depth_rays_preds = depth_rays_preds[H_crop:-H_crop,W_crop:-W_crop]
            img_vis = np.concatenate((rgb_rays*255,depth_rays_preds),axis=1)
            

            frames.append((rgb_rays*255).astype('uint8'))
                


    imageio.mimwrite(f'{save_dir}/{scene}_spiral.mp4', np.stack(frames), fps=10, quality=10)


Found ckpts ['./ckpts//mvsnerf-v0.tar']
Reloading from ./ckpts//mvsnerf-v0.tar
62 62 /mnt/new_disk_2/anpei/Dataset/MVSNeRF/nerf_llff_data/horns
===> valing index: [24 34 13 42]


/home/anpei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
100%|██████████| 40/40 [01:21<00:00,  2.03s/it]


In [20]:
for i_scene, scene in enumerate(['lego']):#'ship','mic','chair','lego','drums','ficus','materials','hotdog'

    cmd = f'--datadir /mnt/new_disk_2/anpei/Dataset/nerf_synthetic/{scene}\
     --dataset_name blender --white_bkgd --imgScale_test {1.0} '

    is_finetued = False # set True if rendering with finetuning
    if is_finetued:
        cmd += f'--ckpt ./runs_fine_tuning/{scene}/ckpts//latest.tar'
    else:
        cmd += '--ckpt ./ckpts//mvsnerf-v0.tar'
        
    args = config_parser(cmd.split())
    args.use_viewdirs = True

    args.N_samples = 128
    args.feat_dim = 8+3*4
    args.use_color_volume = False if not is_finetued else args.use_color_volume

    # create models
    if 0==i_scene:
        render_kwargs_train, render_kwargs_test, start, grad_vars = create_nerf_mvs(args, use_mvs=True, dir_embedder=False, pts_embedder=True)
        filter_keys(render_kwargs_train)

        MVSNet = render_kwargs_train['network_mvs']
        render_kwargs_train.pop('network_mvs')


    datadir = args.datadir
    datatype = 'val'
    pad = 16
    args.chunk = 5120
    frames = 40


    dataset = dataset_dict[args.dataset_name](args, split=datatype)
    val_idx = dataset.img_idx
    
    save_as_image = False
    save_dir = f'results/test3'
    os.makedirs(save_dir, exist_ok=True)
    MVSNet.train()
    MVSNet = MVSNet.cuda()
    
    with torch.no_grad():

        if is_finetued:   
            # large baselien
            imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(device=device)
            volume_feature = torch.load(args.ckpt)['volume']['feat_volume']
            volume_feature = RefVolume(volume_feature.detach()).cuda()
            c2ws_render = nerf_video_path(pose_source['c2ws'], N_views=frames)
        else:            
            # neighboring views with angle distance
            c2ws_all = dataset.load_poses_all()
            random_selete = torch.randint(0,len(c2ws_all),(1,))     #!!!!!!!!!! you may change this line if rendering a specify view 
            dis = np.sum(c2ws_all[:,:3,2] * c2ws_all[[random_selete],:3,2], axis=-1)
            pair_idx = np.argsort(dis)[::-1][torch.randperm(5)[:3]]
            imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(pair_idx=pair_idx, device=device)
            volume_feature, _, _ = MVSNet(imgs_source, proj_mats, near_far_source, pad=pad)
            
            #####
            # still a bug in the gen_render_path
            # you may re-run it if the video looks abnormal ...
            c2ws_render = gen_render_path(c2ws_all[pair_idx], N_views=10)
            c2ws_render = torch.from_numpy(np.stack(c2ws_render)).float().to(device)
            
        imgs_source = unpreprocess(imgs_source)
       
        try:
            tqdm._instances.clear() 
        except Exception:     
            pass
        
        frames = []
        img_directions = dataset.directions.to(device)
        for i, c2w in enumerate(tqdm(c2ws_render)):
            torch.cuda.empty_cache()
            
            rays_o, rays_d = get_rays(img_directions, c2w)  # both (h*w, 3)
            rays = torch.cat([rays_o, rays_d,
                     near_far_source[0] * torch.ones_like(rays_o[:, :1]),
                     near_far_source[1] * torch.ones_like(rays_o[:, :1])],
                    1).to(device)  # (H*W, 3)
            
        
            N_rays_all = rays.shape[0]
            rgb_rays, depth_rays_preds = [],[]
            for chunk_idx in range(N_rays_all//args.chunk + int(N_rays_all%args.chunk>0)):

                xyz_coarse_sampled, rays_o, rays_d, z_vals = ray_marcher(rays[chunk_idx*args.chunk:(chunk_idx+1)*args.chunk],
                                                    N_samples=args.N_samples)

                # Converting world coordinate to ndc coordinate
                H, W = imgs_source.shape[-2:]
                inv_scale = torch.tensor([W - 1, H - 1]).to(device)
                w2c_ref, intrinsic_ref = pose_source['w2cs'][0], pose_source['intrinsics'][0].clone()
                xyz_NDC = get_ndc_coordinate(w2c_ref, intrinsic_ref, xyz_coarse_sampled, inv_scale,
                                             near=near_far_source[0], far=near_far_source[1], pad=pad*args.imgScale_test)


                # rendering
                rgb, disp, acc, depth_pred, alpha, extras = rendering(args, pose_source, xyz_coarse_sampled,
                                                                       xyz_NDC, z_vals, rays_o, rays_d,
                                                                       volume_feature,imgs_source, **render_kwargs_train)
    
                
                rgb, depth_pred = torch.clamp(rgb.cpu(),0,1.0).numpy(), depth_pred.cpu().numpy()
                rgb_rays.append(rgb)
                depth_rays_preds.append(depth_pred)

            
            depth_rays_preds = np.concatenate(depth_rays_preds).reshape(H, W)
            depth_rays_preds, _ = visualize_depth_numpy(depth_rays_preds, near_far_source)
            
            rgb_rays = np.concatenate(rgb_rays).reshape(H, W, 3)
            H_crop, W_crop = np.array(rgb_rays.shape[:2])//20
            rgb_rays = rgb_rays[H_crop:-H_crop,W_crop:-W_crop]
            depth_rays_preds = depth_rays_preds[H_crop:-H_crop,W_crop:-W_crop]
            img_vis = np.concatenate((rgb_rays*255,depth_rays_preds),axis=1)

            frames.append((rgb_rays*255).astype('uint8'))
                

    imageio.mimwrite(f'{save_dir}/{scene}_spiral.mp4', np.stack(frames), fps=10, quality=10)

Found ckpts ['./ckpts//mvsnerf-v0.tar']
Reloading from ./ckpts//mvsnerf-v0.tar
===> valing index: [28, 63, 70, 18]


100%|██████████| 30/30 [04:13<00:00,  8.44s/it]


In [30]:
for i_scene, scene in enumerate([1]):# any scene index, like 1,2,3...

    cmd = f'--datadir /mnt/data/new_disk/sungx/data/mvs_dataset/DTU/mvs_training/dtu/scan{scene} \
     --dataset_name dtu_ft --imgScale_test {1.0} --use_color_volume ' #--use_color_volume
    
    is_finetued = False # set False if rendering without finetuning
    if is_finetued:
        cmd += f'--ckpt ./runs_fine_tuning/dtu_scan{scene}/ckpts//latest.tar'
    else:
        cmd += '--ckpt ./ckpts/mvsnerf-v0.tar'

    args = config_parser(cmd.split())
    args.use_viewdirs = True

    args.N_samples = 128
    args.feat_dim =  8+3*4
    args.use_color_volume = False if not is_finetued else args.use_color_volume


    # create models
    if 0==i_scene:
        render_kwargs_train, render_kwargs_test, start, grad_vars = create_nerf_mvs(args, use_mvs=True, dir_embedder=False, pts_embedder=True)
        filter_keys(render_kwargs_train)

        MVSNet = render_kwargs_train['network_mvs']
        render_kwargs_train.pop('network_mvs')


    datadir = args.datadir
    datatype = 'val'
    pad = 24
    args.chunk = 5120
    frames = 42


    dataset = dataset_dict[args.dataset_name](args, split=datatype)
    val_idx = dataset.img_idx
    
    save_as_image = False
    save_dir = f'results/test3'
    os.makedirs(save_dir, exist_ok=True)
    MVSNet.train()
    MVSNet = MVSNet.cuda()
    
    with torch.no_grad():
        
        if is_finetued:   
            # large baselien
            imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(device=device)
            volume_feature = torch.load(args.ckpt)['volume']['feat_volume']
            volume_feature = RefVolume(volume_feature.detach()).cuda()
        else:            
            # neighboring views with angle distance
            c2ws_all = dataset.read_poses_all()
            random_selete = 11#torch.randint(0,len(c2ws_all),(1,)) #!!!!!!!!!! you may change this line if rendering a specify view 
            dis = np.sum(c2ws_all[:,:3,2] * c2ws_all[[random_selete],:3,2], axis=-1)
            pair_idx = np.argsort(dis)[::-1][:3]
            imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(pair_idx=pair_idx, device=device)
            volume_feature, _, _ = MVSNet(imgs_source, proj_mats, near_far_source, pad=pad)
            
        imgs_source = unpreprocess(imgs_source)

        c2ws_render = gen_render_path(pose_source['c2ws'].cpu().numpy(), N_views=frames//3)
        c2ws_render = torch.from_numpy(np.stack(c2ws_render)).float().to(device)
        
        try:
            tqdm._instances.clear() 
        except Exception:     
            pass
        
        frames = []
        img_directions = dataset.directions.to(device)
        for i, c2w in enumerate(tqdm(c2ws_render)):
            torch.cuda.empty_cache()
            
            rays_o, rays_d = get_rays(img_directions, c2w)  # both (h*w, 3)
            rays = torch.cat([rays_o, rays_d,
                     near_far_source[0] * torch.ones_like(rays_o[:, :1]),
                     near_far_source[1] * torch.ones_like(rays_o[:, :1])],
                    1).to(device)  # (H*W, 3)
            
        
            N_rays_all = rays.shape[0]
            rgb_rays, depth_rays_preds = [],[]
            for chunk_idx in range(N_rays_all//args.chunk + int(N_rays_all%args.chunk>0)):

                xyz_coarse_sampled, rays_o, rays_d, z_vals = ray_marcher(rays[chunk_idx*args.chunk:(chunk_idx+1)*args.chunk],
                                                    N_samples=args.N_samples)

                # Converting world coordinate to ndc coordinate
                H, W = imgs_source.shape[-2:]
                inv_scale = torch.tensor([W - 1, H - 1]).to(device)
                w2c_ref, intrinsic_ref = pose_source['w2cs'][0], pose_source['intrinsics'][0].clone()
                xyz_NDC = get_ndc_coordinate(w2c_ref, intrinsic_ref, xyz_coarse_sampled, inv_scale,
                                             near=near_far_source[0], far=near_far_source[1], pad=pad*args.imgScale_test)


                # rendering
                rgb, disp, acc, depth_pred, alpha, extras = rendering(args, pose_source, xyz_coarse_sampled,
                                                                       xyz_NDC, z_vals, rays_o, rays_d,
                                                                       volume_feature,imgs_source, **render_kwargs_train)
    
                
                rgb, depth_pred = torch.clamp(rgb.cpu(),0,1.0).numpy(), depth_pred.cpu().numpy()
                rgb_rays.append(rgb)
                depth_rays_preds.append(depth_pred)

            
            depth_rays_preds = np.concatenate(depth_rays_preds).reshape(H, W)
            depth_rays_preds, _ = visualize_depth_numpy(depth_rays_preds, near_far_source)
            
            rgb_rays = np.concatenate(rgb_rays).reshape(H, W, 3)
#             H_crop, W_crop = np.array(rgb_rays.shape[:2])//20
#             rgb_rays = rgb_rays[H_crop:-H_crop,W_crop:-W_crop]
#             depth_rays_preds = depth_rays_preds[H_crop:-H_crop,W_crop:-W_crop]
            img_vis = np.concatenate((rgb_rays*255,depth_rays_preds),axis=1)
            
            frames.append((rgb_rays*255).astype('uint8'))
                

    imageio.mimwrite(f'{save_dir}/scan{scene}_spiral.mp4', np.stack(frames), fps=20, quality=10)

Found ckpts ['./ckpts/mvsnerf-v0.tar']
Reloading from ./ckpts/mvsnerf-v0.tar
==> image down scale: 1.0
===> valing index: [32, 24, 23, 44]


100%|██████████| 42/42 [03:01<00:00,  4.32s/it]


# render path generation

In [ ]:
render_poses = {}
datatype = 'val'
for i_scene, scene in enumerate(['flower','orchids', 'room','leaves','fern','horns','trex','fortress']):
    # add --use_color_volume if the ckpts are fintuned with this flag
    cmd = f'--datadir /mnt/new_disk_2/anpei/Dataset/MVSNeRF/nerf_llff_data/{scene}  \
     --dataset_name llff --imgScale_test {1.0} \
    --ckpt ./runs_new/runs_fine_tuning/{scene}/ckpts//latest.tar'

    args = config_parser(cmd.split())
    args.use_viewdirs = True


    print('============> rendering dataset <===================')
    dataset = dataset_dict[args.dataset_name](args, split=datatype)
    val_idx = dataset.img_idx
    

    imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(device=device)

    c2ws_all = dataset.poses
    w2cs, c2ws = pose_source['w2cs'], pose_source['c2ws']
    c2ws_render = llff_video_path(w2cs, c2ws, c2ws_all, near_far_source)
    
    render_poses[f'{scene}_c2ws'] = c2ws_render
    render_poses[f'{scene}_intrinsic'] = pose_source['intrinsics'][0]
    
#######################################
for i_scene, scene in enumerate(['ship','mic','chair','lego','drums','ficus','materials','hotdog']):#

    cmd = f'--datadir /mnt/new_disk_2/anpei/Dataset/nerf_synthetic/{scene}  \
     --dataset_name blender --white_bkgd --imgScale_test {1.0}  \
    --ckpt /mnt/new_disk_2/anpei/code/MVS-NeRF/runs_fine_tuning/{scene}/ckpts//latest.tar '

    args = config_parser(cmd.split())


    print('============> rendering dataset <===================')
    dataset = dataset_dict[args.dataset_name](args, split=datatype)


    imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(device=device)
    c2ws_render = nerf_video_path(pose_source['c2ws'], N_views=120)
    
    render_poses[f'{scene}_c2ws'] = c2ws_render
    render_poses[f'{scene}_intrinsic'] = pose_source['intrinsics'][0]
    
##################################################
for i_scene, scene in enumerate([114]):

    cmd = f'--datadir /mnt/data/new_disk/sungx/data/mvs_dataset/DTU/mvs_training/dtu/scan{scene}  \
     --dataset_name dtu_ft --imgScale_test {1.0}   \
    --ckpt /mnt/new_disk_2/anpei/code/MVS-NeRF/runs_fine_tuning/dtu_scan{scene}/ckpts//latest.tar --netwidth 256 --net_type v0 --use_color_volume'

    args = config_parser(cmd.split())
    args.use_viewdirs = True


    print('============> rendering dataset <===================')
    dataset = dataset_dict[args.dataset_name](args, split=datatype)
    val_idx = dataset.img_idx

    imgs_source, proj_mats, near_far_source, pose_source = dataset.read_source_views(device=device)
    imgs_source = unpreprocess(imgs_source)

    c2ws_render = gen_render_path(pose_source['c2ws'].cpu().numpy(), N_views=40)
    render_poses[f'dtu_c2ws'] = c2ws_render
    render_poses[f'dtu_intrinsic'] = pose_source['intrinsics'][0]
    
torch.save(render_poses, './configs/video_path.th')